In [2]:
from train_inception import feature_extraction_InV3

/Users/Yichen/anaconda2/envs/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
img_width=299
img_height = 299
train_data_dir = "all_image_final/"
test_data_dir="data/test"
num_image=1800
epochs = 10

In [4]:
X_train,y_train,model=feature_extraction_InV3(img_width, img_height,
                        train_data_dir,
                        num_image,
                        epochs)

Found 1800 images belonging to 4 classes.
120/120 [==============================] - 860s 7s/step
(1800, 2048) (1800, 4)


array([0, 0, 0, ..., 3, 3, 3])

# Train Logistic Model

In [9]:
from sklearn.linear_model import LogisticRegression

In [10]:
lg_model=LogisticRegression(multi_class="multinomial",solver="lbfgs",max_iter=4000)

In [13]:
lg_model.fit(X_train,y_train.argmax(axis=1))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=4000, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

# Evaluate Logistic Model

In [36]:
from sklearn.model_selection import cross_val_score, GridSearchCV,StratifiedKFold

In [16]:
loss_scores_lg = cross_val_score(lg_model, X_train,y_train.argmax(axis=1),cv=5,scoring='neg_log_loss')

In [17]:
scores_acc_lg=cross_val_score(lg_model,X_train,y_train.argmax(axis=1),cv=5)

In [18]:
loss_scores_lg.mean()

-0.4916992176852567

In [19]:
scores_acc_lg.mean()

0.846111111111111

# LOGISTIC MODEL TUNNING

In [21]:
parameters = {'C':[1,10,50,100]}
clf = GridSearchCV(lg_model, parameters,scoring='neg_log_loss')
clf.fit(X_train,y_train.argmax(axis=1))
print (clf.best_params_)

{'C': 1}


# Train GBC model

In [22]:
from sklearn.ensemble import GradientBoostingClassifier

In [23]:
gbc_model=GradientBoostingClassifier(learning_rate=0.05,n_estimators=800,
                                    max_depth=3,subsample=0.5,max_features='auto',verbose=1)

In [24]:
gbc_model.fit(X_train,y_train.argmax(axis=1))

      Iter       Train Loss      OOB Improve   Remaining Time 
         1        1181.0089          47.6549           13.80m
         2        1129.3944          40.1024           13.26m
         3        1081.2473          35.1313           13.01m
         4        1042.4116          34.2153           12.83m
         5        1004.0056          32.4823           12.66m
         6         962.8129          30.1733           12.50m
         7         922.0893          25.2924           12.54m
         8         901.5954          24.0733           12.36m
         9         863.7018          21.4796           12.28m
        10         836.8827          22.6573           12.21m
        20         630.7111          12.2460           11.88m
        30         505.4501           6.6884           11.45m
        40         417.1423           4.5486           11.11m
        50         348.4664           2.5796           10.89m
        60         300.2507           1.7621           10.88m
       

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.05, loss='deviance', max_depth=3,
              max_features='auto', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=800,
              presort='auto', random_state=None, subsample=0.5, verbose=1,
              warm_start=False)

In [86]:
parameters = {'max_depth':[3,5]}
clf = GridSearchCV(my_model2, parameters,scoring='neg_log_loss')
clf.fit(X_train,y_train.argmax(axis=1))
print (clf.best_params_)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1         739.8094          58.6577            1.83m
         2         673.2875          48.5734            1.81m
         3         623.5545          41.5884            1.80m
         4         577.3329          36.1885            1.80m
         5         539.4282          28.4541            1.79m
         6         494.6196          26.8402            1.78m
         7         476.8754          23.0570            1.77m
         8         446.9807          18.1031            1.76m
         9         412.7416          17.1357            1.75m
        10         395.4369          15.0389            1.75m
        20         244.4222           5.1527            1.64m
        30         160.4001           2.4965            1.54m
        40         123.0985           0.5649            1.45m
        50          90.8184           0.7224            1.36m
        60          74.1919           0.2989            1.27m
       

         7         742.6551          36.5268            3.07m
         8         690.3067          29.7356            3.02m
         9         660.1904          27.7796            2.97m
        10         626.7575          20.1453            3.03m
        20         411.3904           8.4889            2.74m
        30         303.2012           2.7336            2.53m
        40         224.9165           2.0268            2.36m
        50         183.8837           0.4254            2.19m
        60         160.5803           0.3970            2.04m
        70         132.7830           0.6957            1.88m
        80         109.5016          -0.2291            1.73m
        90          93.0890          -0.0715            1.58m
       100          80.3068          -0.0150            1.43m
       200          20.9940           0.0583            0.00s
{'max_depth': 3}


In [25]:
loss_scores_GBC= cross_val_score(gbc_model, X_train,y_train.argmax(axis=1),cv=5,scoring='neg_log_loss')

      Iter       Train Loss      OOB Improve   Remaining Time 
         1         942.2228          38.0936            9.21m
         2         902.7338          33.2533            9.04m
         3         862.3998          28.7308            8.97m
         4         826.9894          26.8498            9.31m
         5         800.2476          26.4855            9.25m
         6         764.2809          24.0605            9.17m
         7         734.8693          21.2154            9.14m
         8         705.1549          21.6638            9.09m
         9         683.8947          18.3034            9.07m
        10         659.1562          16.8299            9.01m
        20         483.5104           7.4554            8.90m
        30         394.2466           5.6066            8.65m
        40         307.9983           3.5467            8.49m
        50         247.0959           1.1508            8.33m
        60         207.0067           0.6126            8.23m
       

       700           0.6733           0.0009            1.16m
       800           0.3111          -0.0002            0.00s


In [26]:
scores_acc_GBC = cross_val_score(gbc_model, X_train,y_train.argmax(axis=1),cv=5)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1         940.6012          38.6294            9.13m
         2         899.6405          34.3466            9.03m
         3         859.3907          31.2766            9.02m
         4         821.5697          28.5191            8.98m
         5         789.7491          23.3955            8.99m
         6         765.2198          22.7179            8.96m
         7         743.5247          22.0562            8.94m
         8         705.0134          19.6982            8.95m
         9         687.6270          18.2089            8.91m
        10         661.9183          16.8484            8.91m
        20         483.6423           7.7229            8.95m
        30         388.5985           6.0148            9.31m
        40         299.6932           2.7900            9.29m
        50         246.4680           1.5163            9.12m
        60         212.7034           1.1790            9.00m
       

       700           0.6578          -0.0001            1.19m
       800           0.3033           0.0002            0.00s


In [37]:
loss_scores_GBC.mean()

-0.5898052036963848

In [38]:
scores_acc_GBC.mean()

0.8166666666666667

# add relu layer

In [33]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.layers import Dense,GlobalAveragePooling2D,BatchNormalization
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras import backend as K
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping
import numpy as np
import pickle as pk
from keras import regularizers
from sklearn.metrics import accuracy_score,log_loss

In [40]:
from sklearn.metrics import log_loss,accuracy_score

In [41]:
skf = StratifiedKFold(n_splits=5,shuffle=True)

In [43]:
ll=[]
accu=[]
for train_index, test_index in skf.split(X_train, y_train.argmax(axis=1)):
    my_model = Sequential()
    my_model.add(BatchNormalization(input_shape=X_train.shape[1:]))
    my_model.add(Dense(1024, activation = "relu"))
    my_model.add(Dense(4, activation='softmax'))
    my_model.compile(optimizer="SGD", loss='categorical_crossentropy',metrics=['accuracy'])
    my_model.fit(X_train[train_index], y_train[train_index],epochs=18,batch_size=30,verbose=1)
    
    prediction=my_model.predict(X_train[test_index])
    logloss=log_loss(y_train[test_index],prediction)
    ll.append(logloss)
    acc=accuracy_score(y_train.argmax(axis=1)[test_index],prediction.argmax(axis=1))
    accu.append(acc)

Epoch 1/18
1440/1440 [==============================] - 5s 3ms/step - loss: 0.8000 - acc: 0.6792
Epoch 2/18
1440/1440 [==============================] - 2s 1ms/step - loss: 0.3871 - acc: 0.8646
Epoch 3/18
1440/1440 [==============================] - 2s 1ms/step - loss: 0.2899 - acc: 0.9132
Epoch 4/18
1440/1440 [==============================] - 2s 1ms/step - loss: 0.2041 - acc: 0.9486
Epoch 5/18
1440/1440 [==============================] - 2s 1ms/step - loss: 0.1718 - acc: 0.9528
Epoch 6/18
1440/1440 [==============================] - 2s 1ms/step - loss: 0.1356 - acc: 0.9729
Epoch 7/18
1440/1440 [==============================] - 2s 1ms/step - loss: 0.1126 - acc: 0.9826
Epoch 8/18
1440/1440 [==============================] - 2s 1ms/step - loss: 0.0885 - acc: 0.9889
Epoch 9/18
1440/1440 [==============================] - 2s 1ms/step - loss: 0.0849 - acc: 0.9903
Epoch 10/18
1440/1440 [==============================] - 2s 1ms/step - loss: 0.0759 - acc: 0.9924A: 0s - loss: 0.0768 -
Epoch 1

1440/1440 [==============================] - 2s 1ms/step - loss: 0.0690 - acc: 0.9910
Epoch 12/18
1440/1440 [==============================] - 2s 1ms/step - loss: 0.0644 - acc: 0.9931
Epoch 13/18
1440/1440 [==============================] - 2s 1ms/step - loss: 0.0534 - acc: 0.9972
Epoch 14/18
1440/1440 [==============================] - 2s 1ms/step - loss: 0.0539 - acc: 0.9986
Epoch 15/18
1440/1440 [==============================] - 2s 1ms/step - loss: 0.0489 - acc: 0.9958
Epoch 16/18
1440/1440 [==============================] - 2s 1ms/step - loss: 0.0427 - acc: 0.9979
Epoch 17/18
1440/1440 [==============================] - 2s 1ms/step - loss: 0.0428 - acc: 0.9986
Epoch 18/18
1440/1440 [==============================] - 2s 1ms/step - loss: 0.0381 - acc: 0.9993


In [44]:
log_loss=sum(ll)/len(ll)

In [45]:
accuracy=sum(accu)/len(accu)

# Random Forest

In [48]:
from sklearn.ensemble import RandomForestClassifier

In [49]:
RF_model=RandomForestClassifier(n_estimators=700,
                                    max_depth=5,verbose=1)

In [50]:
RF_model.fit(X_train,y_train.argmax(axis=1))

[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:   19.9s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=700, n_jobs=1,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False)

In [173]:
parameters = {'max_depth':[2,3,5]}
clf = GridSearchCV(my_model4, parameters,scoring='neg_log_loss')
clf.fit(X_train,y_train)
print (clf.best_params_)

[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    5.6s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    5.6s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    6.9s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:   10.4s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    8.0s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]

{'max_depth': 5}


[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:   19.0s finished


In [193]:
loss_scores_RF = cross_val_score(RF_model, X_train,y_train,cv=5,scoring='neg_log_loss')

[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:   14.5s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:   14.8s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:   14.1s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:   15.1s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:   14.4s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    0.1s finished


In [194]:
scores_acc_RF = cross_val_score(RF_model, X_train,y_train,cv=5)

[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:   14.7s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:   15.1s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:   15.0s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:   14.9s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:   14.7s finished
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    0.1s finished


In [195]:
loss_scores_RF.mean()

-0.8834106430150653

In [46]:
scores_acc_RF.mean()

NameError: name 'scores_acc_RF' is not defined